Can follow some code in timm's dem merging function.
https://github.com/accs-uaa/beringian-vegetation/blob/master/package_GeospatialProcessing/mergeElevationTiles.py

Recommended workflow for geoprocessing:
- import arcpy
- set environments
- create lists of gis objects
- iterate through list elements
- execute geoprocessing tools

Geoprocessing steps in this script
1. create a set of folders and gdb for each region under a master AKSSF folder
2. loop through regions and grids or feature classes (e.g. catchments or flowlines)
3. merge grids or feature classes
4. save in folder/gdb with consistent name (e.g. fac_merge)

These seamed grids will enable us to use zonal statistics with the selected catchments
and watersheds to get our environmental variables for modeling.

In [1]:
# step 1
# Create folders for storing outputs

#import modules
import os, arcpy, sys
#user inputs
baseFolder = "D:\\GIS\\"
projectName = "AKSSF"
akssfol = os.path.join(baseFolder, projectName)

#define folder paths
ciFolder = baseFolder + "\\" + projectName + "\\"+ "Cook_Inlet"
pwsFolder = baseFolder + "\\" + projectName + "\\"+ "Prince_William_Sound"
bbFolder = baseFolder + "\\" + projectName + "\\"+ "Bristol_Bay"
crFolder = baseFolder + "\\" + projectName + "\\"+ "Copper_River"
kodFolder = baseFolder + "\\" + projectName + "\\"+ "Kodiak"

#create list of folder paths
folders = [ciFolder, pwsFolder, bbFolder, crFolder, kodFolder]

#check if project folder exists
if not os.path.exists(baseFolder + projectName):
    #Create folder if it does not exist
    for folder in folders:
        os.makedirs(folder)
        print("Createds: " + folder)
else:
    #if project folder exists, print error message
    print (projectName + " exists. Please provide a new project name.")

AKSSF exists. Please provide a new project name.


In [19]:
# step 2
# merge catchments, waterbodies, flowlines,  and vaa tables
# must copy all flowlines and waterbodies to scratch gdb before merging or the operation will fail.
#This code chunk is for NHDPlus regions -- Cook Inlet and Copper River

import arcpy, sys, os
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

#scratch gdb to store copies of nhdplus data that will not merge
scratch = r'D:\GIS\scratch.gdb'
#path to folder with most Current NHdplus data split by region
nhdPlusfol = r'D:\Basedata\NHDPlus08112021'
rois = ['Cook_Inlet','Copper_River']

#NHD folders - Update to new data downloaded Nov 8, 2021
for roi in rois:
    arcpy.env.workspace = os.path.join(nhdPlusfol,roi)
    gdbs = arcpy.ListWorkspaces('NHDPLUS*','FileGDB')
    print(gdbs)
    Cats = []
    VAA = []
    waterbodies = []
    flowlines = []

    outpath =os.path.join(akssfol, roi)
    gdbname = roi + '.gdb'
    outgdb = os.path.join(outpath, gdbname)
    if not arcpy.Exists(outgdb):
        print(f'Creating {outgdb}')
        arcpy.CreateFileGDB_management(outpath, roi+'.gdb ')
    else:
        print(f'{outgdb} already exists')
        outgdb = outgdb

    for gdb in gdbs:
        arcpy.env.workspace = gdb
        huc = gdb[-20:-12]
        datasets = arcpy.ListDatasets(feature_type='feature')
        for ds in datasets:
            for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
                if fc == "NHDFlowline":
                    flowpath = os.path.join(arcpy.env.workspace, ds, fc)
                    flowname = 'flowline_' + str(huc)
                    print(f'Copying {flowname}...')
                    flowcopypath = os.path.join(scratch, flowname)
                    arcpy.management.CopyFeatures(flowpath, flowcopypath)
                    flowlines.append(flowcopypath)
                elif fc == "NHDWaterbody":
                    waterbodyname = 'waterbody_' + str(huc)
                    waterbodycopypath = os.path.join(scratch, waterbodyname)
                    waterpath = os.path.join(arcpy.env.workspace, ds, fc)
                    print(f'Copying {waterbodyname}...')
                    arcpy.management.CopyFeatures(waterpath, waterbodycopypath)
                    waterbodies.append(waterbodycopypath)
                elif fc == "NHDPlusCatchment":
                    catchpath = os.path.join(arcpy.env.workspace, ds, fc)
                    Cats.append(catchpath)
        vaapath = os.path.join(arcpy.env.workspace, "NHDPlusFlowlineVAA")
        VAA.append(vaapath)
    arcpy.env.workspace =  outgdb
    arcpy.env.outputCoordinateSystem = sr
    arcpy.env.overwriteOutput = True
    try:
        print(f'Merging catchments for {roi}...')
        print('----------')
        arcpy.management.Merge(Cats, os.path.join(outgdb,'cats_merge'))
        print(f'Merging vaas for {roi}...')
        print('----------')
        arcpy.management.Merge(VAA, os.path.join(outgdb,'vaa_merge'))
        print(f'Merging flowlines for {roi}...')
        print('----------')
        arcpy.management.Merge(flowlines, os.path.join(outgdb,'NHDFlowline_merge'))
        print(f'Merging waterbodies for {roi}...')
        print('----------')
        arcpy.management.Merge(waterbodies, os.path.join(outgdb,'NHDWaterbody_merge'))
    except:
        e = sys.exc_info()[1]
        print(e.args[0])
        arcpy.AddError(e.args[0])

print('NHDPlus gdb data merge complete')


['D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020302_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020401_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020402_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020501_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020502_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020503_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020504_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020505_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020601_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020602_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus08112021\\Cook_Inlet\\NHDPLUS_H_19020800_HU8_GDB.gdb'

# Merge catchments and streams for Bristol Bay

This code chunk is for Bristol Bay, which had several processing units created
with synthetic networks from TauDEM. The gridcodes all start from 1 so are not unique
when combining the different VPUS. To create unique IDs across the VPUs,
need to make a new catID with a prefix for each VPU: 10, 20, 30, 40, 50.
Note that gridcode = LINKNO in the stream reach feature class.
Also for the streams fc that we are using to create watersheds, the upstream
links must be fixed so they are unique as well.

For Kodiak and Prince William Sound, can just copy over catchments and streams since
only one feature class for each.

In [ ]:
import arcpy
import os
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
baseFolder = "D:\\GIS\\"
ds = baseFolder + "AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs"
region = "Bristol_Bay"
arcpy.env.workspace = ds
cats = arcpy.ListFeatureClasses('Catchments*')
streams = arcpy.ListFeatureClasses('Stream*')

#For Bristol Bay - remove feature classes with Pws
cats = [x for x in cats if 'Pws' not in x]
streams = [x for x in streams if 'Pws' not in x]

#get full path names onto lists
cats = [ds + "\\" + x for x in cats]
streams = [ds + "\\" + x for x in streams]

#add a new field with the VPU prefix to the gridcode.
# for cat in cats:
#     if len(arcpy.ListFields(cat, "catID")) > 0:
#         arcpy.DeleteField_management(cat, "catID")
#         arcpy.AddField_management(cat, "catID", "LONG")
#     vpu = cat[61:63]
#     print(vpu)
#     options = {'Eg': 1000000, 'La': 2000000, 'Na': 3000000, 'Nu': 4000000, 'To': 5000000}
#     if vpu in options:
#         prefix = options[vpu]
#     else:
#         print("vpu not found")
#     arcpy.management.CalculateField(cat, "catID", 'prefix + !gridcode!')

# for stream in streams:
#     print(stream)
#     if len(arcpy.ListFields(stream, "catID")) > 0:
#         arcpy.DeleteField_management(stream, ["catID", "upCatID1", "upCatID2"])
#         arcpy.management.AddFields(stream,[["catID", "LONG"], ["upCatID1", "LONG"], ["upCatID2", "LONG"]])
#     vpu = stream[65:67]
#     print(vpu)
#     # options = {'Eg': 1000000, 'La': 2000000, 'Na': 3000000, 'Nu': 4000000, 'To': 5000000}
#     if vpu in options:
#         prefix = options[vpu]
#     else:
#         print("vpu not found")
#     arcpy.management.CalculateField(stream, "catID", 'prefix + !LINKNO!')
#     arcpy.management.CalculateField(stream, "upCatID1", 'prefix + !USLINKNO1!')
#     arcpy.management.CalculateField(stream, "upCatID2", 'prefix + !USLINKNO2!')


print("Catchment feature classes to be merged: ")
print(cats)
print("Stream feature classes to be merged: ")
print(streams)

arcpy.env.outputCoordinateSystem = arcpy.Describe(cats[1]).spatialReference
arcpy.env.overwriteOutput = True
arcpy.CreateFileGDB_management(baseFolder + "AKSSF\\" + region, region + ".gdb")
# arcpy.env.workspace =  r"W:/GIS/AKSSF/" + region + ".gdb"

arcpy.management.Merge(cats, r"W:/GIS/AKSSF/" + region + "\\" + region + ".gdb/cats_merge")
arcpy.management.Merge(streams, r"W:/GIS/AKSSF/" + region + "\\" + region + ".gdb/streams_merge")

print("Merged feature classes saved to: " + arcpy.env.workspace)


# Copy feature classes for PWS and Kodiak (one VPU each)

Copy feature classes from AKSSF Hydrography geodb to new region specific geodb
for Kodiak and PWS. These will be used to create watersheds and run zonal
stats for catchments and watersheds.

In [ ]:
import arcpy
import os
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

# Create new geodb in each regional folder
baseFolder = "D:\\GIS\\"
region = "Prince_William_Sound"
# region = "Kodiak"

# TauDEM outputs
pwsNet = baseFolder + "AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_Outputs"
kodNet = baseFolder + "AKSSF\\AKSSF_Hydrography.gdb\\TauDEM_StreamBurn"

arcpy.env.workspace = pwsNet
fcs = arcpy.ListFeatureClasses()
fcs = [os.path.join(arcpy.env.workspace, x) for x in fcs]
fcs = [x for x in fcs if "Pws" in x]
print(fcs)

# arcpy.CreateFileGDB_management(r"W:/GIS/AKSSF/" + region, region + ".gdb")
arcpy.env.workspace =  r"W:/GIS/AKSSF/" + region + "//" + region + ".gdb"
arcpy.env.outputCoordinateSystem = arcpy.Describe(fcs[0]).spatialReference
arcpy.env.overwriteOutput = True

# Copy feature classes over from AKSSF_Hydrography geodb
for fc in fcs:
    if "Cat" in fc:
        out_fc = "cats_merge"
    else:
        out_fc = "streams_merge"
    arcpy.CopyFeatures_management(fc, out_fc)

# Merge raster datasets by region
1. Create a regional boundary (maybe buffer by 10 km) for clipping statewide datasets and putting all grids into regional geodb
2. Elevation grids: clip 10m DEM for synthetic flow networks (ask Dustin for this) and merge elevation rasters for each NHDPlus VPU
3. Flow accumulation grids: copy and/or merge as needed by region
4. Use elevation grids to create slope grids for each region
5. Use elevation grids to create the aspect grid
6. Use elevation grids to create a grid of north facing cells (Dustin has this code ready)
7. Lake feature class: merge from NHPPlus VPUs or clip from regular NHD to region geodb
8. Copy glims glacier inventory shp to local AKSSF folder and clip to create glacier polys for each region
9. Copy NLCD to local AKSSF folder and clip create a 1/0 grid of wetland cover (Dustin should have this code in deshka repo)
10. (MODIS lcld in separate script)

In [2]:
# step 1, create regional boundaries with buffer for clipping grids
# dissolve on cats_merge and buffer by 10 km

import arcpy
import os
arcpy.env.overwriteOutput = True
arcpy.env.workspace = akssfol
regions = arcpy.ListWorkspaces(workspace_type="Folder")
nhdplus_dat = ['Cook_Inlet', 'Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']
# Loop through all processing areas
#rois = nhdplus_dat + tauDem_dat
rois = nhdplus_dat
for roi in rois:
    # Loop through regional folders
    for region in regions:
            if roi in str(region):
                walk = arcpy.da.Walk(region, datatype=['FeatureClass', 'Table'])
                for dirpath, dirnames, filenames in walk:
                   for filename in filenames:
                       if filename == 'cats_merge':
                           catspath = os.path.join(dirpath,filename)
                           print (catspath)
                           print (f'Buffering {catspath}....')
                           arcpy.Buffer_analysis(catspath, os.path.join(dirpath, 'region_buf'), "10 Kilometers", "FULL", "ROUND", "ALL")
print('buffer complete')

D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\cats_merge
Buffering D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\cats_merge....
D:\GIS\AKSSF\Copper_River\Copper_River.gdb\cats_merge
Buffering D:\GIS\AKSSF\Copper_River\Copper_River.gdb\cats_merge....
buffer complete


In [3]:
# steps 2 and 3: merge elevation and flow accumulation rasters for regions with NHDPlus

import arcpy
import os
from arcpy.sa import *
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
arcpy.env.workspace = akssfol
# Let tools decide parallel processing factor
arcpy.env.parallelProcessingFactor = ""
regions = arcpy.ListWorkspaces()
print (regions)
import time
start_time = time.time()

#Path to folder with NHDPlus data separated by region
nhdPlusfol = r'D:\Basedata\NHDPlus08112021'
rois = ['Cook_Inlet','Copper_River']
for roi in rois:
    for region in regions:
        if roi in str(region):
            nhdfol = os.path.join(nhdPlusfol,roi)
            print(f'Processing {roi} using rasters in {nhdfol}...')
            flowacc_rasters = []
            elev_rasters = []
            walk = arcpy.da.Walk(nhdfol, datatype = "RasterDataset")
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    if filename == 'fac.tif':
                        flowacc_raspath = os.path.normpath(os.path.join(dirpath, filename))
                        flowacc_rasters.append(flowacc_raspath)
                    elif filename == 'elev_cm.tif':
                        elev_raspath = os.path.normpath(os.path.join(dirpath, filename))
                        elev_rasters.append(elev_raspath)
            print(f'Merging Flow acc...')
            fac = arcpy.MosaicToNewRaster_management(flowacc_rasters, region, "fac.tif", sr, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
            print('Merging elev_cm...')
            elev_cm = arcpy.MosaicToNewRaster_management(elev_rasters, region, "elev_cm.tif", sr, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
            print('Converting elevation in cm to meters, resampling to 10 meters cellsize and converting to Integer...')
            elev = arcpy.Resample_management(Int(Times(elev_cm, .01)), os.path.join(region, 'elev.tif'), "10", "BILINEAR")

print(f'Merge complete for {rois}')
print('----------')
arcpy.AddMessage("--- %s seconds ---" % (time.time() - start_time))
print ("--- %s seconds ---" % (time.time() - start_time))
print('----------')

Merging elev_cm...
Converting elevation in cm to meters, resampling to 10 meters cellsize and converting to Integer...
Processing Copper_River using rasters in D:\Basedata\NHDPlus08112021\Copper_River...
Merging Flow acc...
Merging elev_cm...
Converting elevation in cm to meters, resampling to 10 meters cellsize and converting to Integer...
Merge complete for ['Cook_Inlet', 'Copper_River']
----------
--- 9057.211299657822 seconds ---
----------
['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']
Processing Cook_Inlet using rasters in D:\Basedata\NHDPlus08112021\Cook_Inlet...
Merging Flow acc...
Merging elev_cm...
Converting elevation in cm to meters, resampling to 10 meters cellsize and converting to Integer...
Processing Copper_River using rasters in D:\Basedata\NHDPlus08112021\Copper_River...
Merging Flow acc...
Merging elev_cm...
Converting elevation in cm to meters, resamplin

In [ ]:
# steps 2 and 3: merge elevation and flow accumulation rasters for regions with NHDPlus

import arcpy
import os
from arcpy.sa import *
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
arcpy.env.workspace = akssfol
# Let tools decide parallel processing factor
arcpy.env.parallelProcessingFactor = ""
regions = arcpy.ListWorkspaces()
print (regions)
import time
start_time = time.time()

#Path to folder with NHDPlus data separated by region
nhdPlusfol = r'D:\Basedata\NHDPlus08112021'
rois = ['Cook_Inlet','Copper_River']
for roi in rois:
    for region in regions:
        if roi in str(region):
            nhdfol = os.path.join(nhdPlusfol,roi)
            print(f'Processing {roi} using rasters in {nhdfol}...')
            flowacc_rasters = []
            elev_rasters = []
            walk = arcpy.da.Walk(nhdfol, datatype = "RasterDataset")
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    if filename == 'fac.tif':
                        flowacc_raspath = os.path.normpath(os.path.join(dirpath, filename))
                        flowacc_rasters.append(flowacc_raspath)
                    elif filename == 'elev_cm.tif':
                        elev_raspath = os.path.normpath(os.path.join(dirpath, filename))
                        elev_rasters.append(elev_raspath)
            print(f'Merging Flow acc...')
            fac = arcpy.MosaicToNewRaster_management(flowacc_rasters, region, "fac.tif", sr, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
            print('Merging elev_cm...')
            elev_cm = arcpy.MosaicToNewRaster_management(elev_rasters, region, "elev_cm.tif", sr, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
            print('Converting elevation in cm to meters, resampling to 10 meters cellsize and converting to Integer...')
            elev = arcpy.Resample_management(Int(Times(elev_cm, .01)), os.path.join(region, 'elev.tif'), "10", "BILINEAR")

print(f'Merge complete for {rois}')
print('----------')
arcpy.AddMessage("--- %s seconds ---" % (time.time() - start_time))
print ("--- %s seconds ---" % (time.time() - start_time))
print('----------')

In [ ]:
# steps 2 and 3: clip elevation grid and copy flow accumulation raster for regions with synthetic networks
# T:\Aquatic\AKSSF\topography for 10 m DEM used on these networks
# This code chunk clips the 10m dem for each region
import arcpy
from arcpy.sa import *

arcpy.env.overwriteOutput = True

dem_sa = "T:\\Aquatic\\AKSSF\\topography\\AKSSF_Elevation_Composite_10m.tif"
dem_bb = "T:\\Aquatic\\AKSSF\\topography\\AKSSF_Bristol_Bay_10m_comp_clip.tif"
arcpy.env.outputCoordinateSystem = dem_sa
arcpy.env.snapRaster = dem_sa

regions = ['Bristol_Bay', 'Prince_William_Sound', 'Kodiak']
base_folder = "W:\\GIS\\AKSSF\\"

for region in regions:
    folder = base_folder + region
    buffer = folder + "\\" + region + ".gdb\\region_buf"
    print(buffer)
    extract_raster = arcpy.sa.ExtractByMask(dem_sa, buffer)
    extract_raster.save(folder + "\\elev.tif")

Copy FAC grids for synthetic networks - ad8 grid from TauDEM
For BB, will need to seam them together, similar to NHDPlus regions


In [ ]:
import os
import arcpy
from arcpy.sa import *

arcpy.env.overwriteOutput = True

baseFolder = "W:\\GIS\\AKSSF\\"
v1_folder = r"W:\GIS\AKSSF\TauDEM\V1"
v2_folder = r"W:\GIS\AKSSF\TauDEM\V2_CartoStreamBurn\KodiakAfognakIslands"

#Bristol Bay first
arcpy.env.workspace = v1_folder
flowacc_rasters = []
raster_folders = arcpy.ListWorkspaces()
condition = ["Kodiak", "Pws", "ResRiver"]
# raster_folders = [x for x in raster_folders if any(c not in raster_folders for c in condition)]
raster_folders = [x for x in raster_folders if not ("Pws" in x or "Kodiak" in x or "ResRiver" in x)]
print(raster_folders)
for folder in raster_folders:
    arcpy.env.workspace = folder
    fac_grids = arcpy.ListRasters("*ad8.tif")
    for grid in fac_grids:
        fac_path = os.path.join(folder, grid)
        print(fac_path)
        flowacc_rasters.append(fac_path)
dataset = flowacc_rasters[1]
spatial_ref = arcpy.Describe(dataset).spatialReference
outFolder = baseFolder + "Bristol_Bay"
print(outFolder)
arcpy.MosaicToNewRaster_management(flowacc_rasters, outFolder, "fac.tif", spatial_ref, "32_BIT_SIGNED", "10", "1", "LAST","FIRST")

#Prince William Sound
fac_path = v1_folder + "\\Pws\\Pws_ad8.tif"
print(fac_path)
out_fac = baseFolder + "Prince_William_Sound\\fac.tif"
arcpy.CopyRaster_management(fac_path, out_fac)

# Kodiak
fac_path = v2_folder + "\\KodiakAfognakIslands_ad8.tif"
print(fac_path)
out_fac = baseFolder + "Kodiak\\fac.tif"
arcpy.CopyRaster_management(fac_path, out_fac)

## Slope, Aspect, and North grids
Calculate slope and aspect with the ESRI recommended surface parameter tool using a 5x5 window.  Produce North grid using
new Aspect raster.

In [4]:
# steps 4-6, use elevation grids for each region to create slope, aspect, and north-facing grids.

import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
arcpy.env.workspace = akssfol
region_folders = arcpy.ListWorkspaces()
#region_folders_filter = [x for x in region_folders if "Bristol_Bay" in x or "Kodiak" in x or "Prince_William_Sound" in x]
#region_folders_filter = [x for x in region_folders if "Cook" in x or "Copper" in x]
# Re-run for all regions using new surface parameters tool and overwrite existing datasets
region_folders_filter = region_folders
print(region_folders_filter)

for folder in region_folders_filter:
    arcpy.env.workspace = folder
    elev_raster = folder + "\\elev.tif"
    print(elev_raster)
    arcpy.env.snapRaster = elev_raster
    aspect2 = arcpy.sa.SurfaceParameters(elev_raster, "ASPECT", "", 5, "", "METER")
    aspect2.save(folder + "\\aspect.tif")
    slope2 = arcpy.sa.SurfaceParameters(elev_raster, "SLOPE", "", 5, "", "METER", "DEGREE")
    slope2.save(folder + "\\slope.tif")
    aspect_raster = folder + "\\aspect.tif"
    aspect_in = Raster(aspect_raster)
    north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
    north_raster.save("north.tif")

['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']
D:\GIS\AKSSF\Bristol_Bay\elev.tif
D:\GIS\AKSSF\Cook_Inlet\elev.tif
D:\GIS\AKSSF\Copper_River\elev.tif
D:\GIS\AKSSF\Kodiak\elev.tif
D:\GIS\AKSSF\Prince_William_Sound\elev.tif


# Glaciers
Downloaded RGI version 6.0 from 2017 locally as that would be the most current
and best source of glacier outlines.
 - No need to re-run this as of Nov 18 2021

In [ ]:
import arcpy
import os
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

baseFolder = baseFolder
rgi = baseFolder + "Randolph_Glacier_Inventory\\01_rgi60_Alaska.shp"
regions = ["Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound", "Kodiak"]
regions = ["Cook_Inlet"]
arcpy.env.overwriteOutput = True

#Note, the glims layer has overlapping polygons, just select the glacier lines for Alaska.
arcpy.env.workspace = baseFolder

for region in regions:
    gdb = baseFolder + region + "\\" + region + ".gdb"
    arcpy.env.workspace = gdb
    reg_buf = os.path.join(gdb, "region_buf")
    print(reg_buf)
    glacier_clip = arcpy.Clip_analysis(rgi, reg_buf, "glaciers")
    print("glaciers created for " + region)

# Wetlands

For wetlands, first convert NLCD to wetlands 1/0 grid, then mask using the study area for each region.
- No need to re-run this as of Nov 18 2021

In [ ]:
import arcpy
import os
from arcpy.sa import *

nlcd = "K:\\GIS_data\\biota\\Terrestrial\\Flora\\ak_nlcd_2011_landcover_1_15_15\\ak_nlcd_2011_landcover_1_15_15.img"
baseFolder = "W:\\GIS\\AKSSF\\"
regions = ["Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound", "Kodiak"]
# regions = [x for x in regions if "Kodiak" in x]

arcpy.env.overwriteOutput = True

nlcd_ras = Raster(nlcd)
wetlands = Con(nlcd_ras == 90, 1, Con(nlcd_ras == 95, 1, 0))

for region in regions:
    gdb = baseFolder + region + "\\" + region + ".gdb"
    reg_buf = os.path.join(gdb, "region_buf")
    print(reg_buf)
    arcpy.env.workspace = baseFolder + region
    wet_mask = arcpy.sa.ExtractByMask(wetlands, reg_buf)
    wet_mask.save(baseFolder + region + "\\wetlands.tif")